# CloudWatch Database Insights Setup and Analysis

<div style="background-color: #f8f9fa; border: 1px solid #e9ecef; border-radius: 8px; padding: 10px; margin: 10px;">
<strong>📋 Workshop Contents</strong>
<ul style="line-height: 1.2;">
<li><a href="#Prerequisites">Prerequisites</a></li>
<li><a href="#Setting-up-the-environment-and-required-libraries">Setting up the environment and required libraries</a></li>
<li><a href="#1-Setting-up-Database-Insights">1. Setting up Database Insights</a></li>
<li><a href="#2-Interpreting-Performance-Recommendations">2. Interpreting Performance Recommendations</a></li>
<li><a href="#3-Implementing-Optimizations">3. Implementing Optimizations</a></li>
<li><a href="#Usage-Example">Usage Example</a></li>
</ul>
</div>

## Prerequisites

Before starting this workshop, ensure you have:

- ✅ **Jupyter Notebook**: You can launch a [free tier Amazon SageMaker Jupyter Notebook](../../1_Getting_Started_with_AWS/1.4_Setting_up_Your_Cookbook_Environment/README.MD)
- ✅ **Aurora PostgreSQL Cluster**: An active Aurora PostgreSQL cluster running in your AWS account
  - If you don't have one, follow the setup guide: [Your First Database on AWS](../../2_Your_First_Database_on_AWS/README.MD)
- ✅ **AWS CLI configured** with appropriate permissions for RDS and Performance Insights
- ✅ **Python environment** with boto3 installed
- ✅ **IAM permissions** for:
  - Performance Insights enablement and access
  - RDS instance modifications
  - CloudWatch metrics access

> 💡 **Note**: Performance Insights provides 7 days of retention for free. Extended retention requires additional charges.

## Setting up the environment and required libraries

This step initializes the required Python libraries and AWS clients for Performance Insights analysis including boto3 for AWS SDK operations, json for data handling, and datetime for time-based metrics collection. You'll see the initialization of RDS, CloudWatch, and Performance Insights clients for comprehensive database monitoring.

In [1]:
import boto3
import json
from datetime import datetime, timedelta

# Initialize AWS clients
rds = boto3.client('rds')
cloudwatch = boto3.client('cloudwatch')
pi = boto3.client('pi')  # Performance Insights client

## 1. Setting up Database Insights

This function enables Performance Insights for Aurora instances, providing detailed database performance monitoring with 7-day retention for the free tier. Performance Insights offers deep visibility into database load, wait events, and SQL statement performance with minimal overhead. You'll see the instance modification response confirming Performance Insights activation.

In [ ]:
%%bash
echo "Setting Cluster Identifier parameter"
export cluster_identifier=<cluster-identifier>

echo "Configuring Performance Insights"
aws rds modify-db-cluster \
    --db-cluster-identifier $cluster_identifier \
    --database-insights-mode advanced \
    --enable-performance-insights \
    --performance-insights-retention-period 465 \
    --apply-immediately

echo "✅ Performance Insights configured successfully"

## 2. Interpreting Performance Recommendations

These functions retrieve and analyze Performance Insights metrics to identify performance bottlenecks and generate actionable recommendations. The analysis focuses on database load patterns and wait events to pinpoint specific areas for optimization. You'll see performance metrics data and automated recommendations based on identified performance patterns.


In [2]:
def get_performance_metrics(resource_id, start_time, end_time):
    """Retrieve performance metrics from Performance Insights"""
    try:
        response = pi.get_resource_metrics(
            ServiceType='RDS',
            Identifier=resource_id,
            StartTime=start_time,
            EndTime=end_time,
            MetricQueries=[
                {
                    'Metric': 'db.load.avg',
                    'GroupBy': {'Group': 'db.wait_event'}
                }
            ]
        )
        return response
    except Exception as e:
        print(f"Error retrieving performance metrics: {str(e)}")
        return None

def analyze_performance_data(metrics_response):
    """Analyze performance data and provide recommendations"""
    try:
        recommendations = []
        if metrics_response and 'MetricList' in metrics_response:
            for metric in metrics_response['MetricList']:
                if metric['Value'] > 5:  # threshold for high load
                    recommendations.append({
                        'event': metric['GroupBy']['Value'],
                        'load': metric['Value'],
                        'suggestion': get_optimization_suggestion(metric['GroupBy']['Value'])
                    })
        return recommendations
    except Exception as e:
        print(f"Error analyzing performance data: {str(e)}")
        return None

## 3. Implementing Optimizations

These functions provide optimization suggestions based on wait event analysis and implement recommended changes including instance scaling and parameter adjustments. The optimization engine maps specific wait events to targeted solutions for improved database performance. You'll see optimization suggestions and implementation responses for applied changes.

In [3]:
def get_optimization_suggestion(wait_event):
    """Get optimization suggestions based on wait events"""
    suggestions = {
        'CPU': 'Consider scaling up instance class or optimizing queries',
        'IO:XactSync': 'Optimize transaction commit patterns',
        'IO:BufFileRead': 'Increase work_mem to reduce disk I/O',
        'LWLock': 'Review and optimize concurrent transactions',
        'Lock': 'Check for lock contentions in transactions'
    }
    return suggestions.get(wait_event, 'Monitor and analyze specific wait event')

def apply_optimization(instance_identifier, optimization_type, params):
    """Apply suggested optimizations to the database"""
    try:
        if optimization_type == 'instance_scaling':
            response = rds.modify_db_instance(
                DBInstanceIdentifier=instance_identifier,
                DBInstanceClass=params['new_instance_class'],
                ApplyImmediately=True
            )
        elif optimization_type == 'parameter_update':
            response = rds.modify_db_parameter_group(
                DBParameterGroupName=params['parameter_group'],
                Parameters=params['parameters']
            )
        return response
    except Exception as e:
        print(f"Error applying optimization: {str(e)}")
        return None

## Usage Example

This comprehensive example demonstrates implementing a complete performance optimization workflow including Performance Insights setup, metrics analysis, recommendation generation, and performance monitoring. The script provides end-to-end performance tuning with automated recommendations and improvement tracking. You'll see performance analysis results, optimization recommendations, and measurable performance improvements.

In [ ]:
# Configuration variables
INSTANCE_IDENTIFIER = 'cluster-identifier'
RESOURCE_ID = 'resource-identifier'
START_TIME = datetime.utcnow() - timedelta(hours=1)
END_TIME = datetime.utcnow()

# 1. Enable Performance Insights
#insights_config = enable_performance_insights(INSTANCE_IDENTIFIER)
print("Enabled Performance Insights")

# 2. Get and analyze performance metrics
metrics = get_performance_metrics(RESOURCE_ID, START_TIME, END_TIME)
recommendations = analyze_performance_data(metrics)
print("Retrieved performance recommendations")

# 3. Apply optimizations based on recommendations
for rec in recommendations:
    print(f"Recommendation for {rec['event']}: {rec['suggestion']}")

# Monitor optimization results
def monitor_performance_changes(instance_identifier, start_time, end_time):
    try:
        # Get performance metrics before and after optimization
        metrics = get_performance_metrics(instance_identifier, start_time, end_time)
        # Calculate improvement
        before_load = sum([m['Value'] for m in metrics['MetricList'][:len(metrics['MetricList'])//2]])
        after_load = sum([m['Value'] for m in metrics['MetricList'][len(metrics['MetricList'])//2:]])
        improvement = ((before_load - after_load) / before_load) * 100 if before_load > 0 else 0
        return {
            'before_load': before_load,
            'after_load': after_load,
            'improvement_percentage': improvement
        }
    except Exception as e:
        print(f"Error monitoring performance changes: {str(e)}")
        return None

# Monitor the results
monitoring_results = monitor_performance_changes(
    INSTANCE_IDENTIFIER,
    START_TIME,
    END_TIME
)

if monitoring_results:
    print(f"Performance improvement: {monitoring_results['improvement_percentage']:.2f}%")

## Next Steps 🚀

Now that you've optimized your Aurora database performance, continue building operational excellence with these next steps:

1. **Set Up Development and Testing Environments**
   - Implement Aurora cloning for testing in [4.5 Aurora Cloning for Testing and Development](../4.5_Aurora_Cloning_for_Testing_and_Development/aurora_cloning_testing_devlop.ipynb)
   - Create isolated environments for safe testing and development

2. **Scale Your Aurora Infrastructure**
   - Learn advanced scaling strategies in [Section 5: Scaling for Success - Growing with Aurora](../../5_Scaling_for_Success_Growing_with_Aurora/README.md)
   - Implement vertical and horizontal scaling techniques

3. **Implement Advanced Features**
   - Explore Aurora Global Database for multi-region deployments
   - Set up Aurora Serverless v2 for automatic scaling

Continue to [4.5 Set Up Development and Testing Environments](../4.5_Aurora_Cloning_for_Testing_and_Development/aurora_cloning_testing_devlop.ipynb).


## Additional Resources 📚

### Performance Insights
- [Performance Insights User Guide](https://docs.aws.amazon.com/AmazonRDS/latest/AuroraUserGuide/USER_PerfInsights.html)
- [Performance Insights API Reference](https://docs.aws.amazon.com/performance-insights/latest/APIReference/)
- [Performance Insights Best Practices](https://docs.aws.amazon.com/AmazonRDS/latest/AuroraUserGuide/USER_PerfInsights.BestPractices.html)

### Aurora Performance Tuning
- [Aurora Performance Best Practices](https://docs.aws.amazon.com/AmazonRDS/latest/AuroraUserGuide/Aurora.BestPractices.html)
- [Aurora MySQL Performance Tuning](https://docs.aws.amazon.com/AmazonRDS/latest/AuroraUserGuide/AuroraMySQL.BestPractices.html)
- [Aurora PostgreSQL Performance Tuning](https://docs.aws.amazon.com/AmazonRDS/latest/AuroraUserGuide/AuroraPostgreSQL.BestPractices.html)

### Monitoring & Analysis
- [CloudWatch Database Insights](https://docs.aws.amazon.com/AmazonCloudWatch/latest/monitoring/CloudWatch-Insights-for-RDS.html)
- [Aurora Monitoring Guide](https://docs.aws.amazon.com/AmazonRDS/latest/AuroraUserGuide/MonitoringAurora.html)
- [Wait Event Analysis](https://docs.aws.amazon.com/AmazonRDS/latest/AuroraUserGuide/USER_PerfInsights.UsingDashboard.html)